# Realworld Study Dashboard

## Imports, Settings and Functions

### Imports

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash.dependencies
import plotly.io as pio
import math
import pysrt
from plotly.subplots import make_subplots
from plotly.graph_objs import *
from scipy.stats import spearmanr
import json

### Settings

In [2]:
# Visuals
pd.options.plotting.backend = "plotly"
external_stylesheets = ['https://bootswatch.com/5/flatly/bootstrap.min.css']
pio.templates.default = "simple_white" #seaborn plotly_dark

#Colors
COLOR_SIT_A_HEX = "#7a5b7b"
COLOR_SIT_B_HEX = "#f9dbbd"
COLOR_SIT_C_HEX = "#fca17d"
COLOR_SIT_D_HEX = "#68534d"
COLOR_SIT_E_HEX = "#5b7b7a"
COLOR_SIT_X_HEX = "#8f2d41"
marking_red = "#961C26"

layout = Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

# Headrotation interpretation Settings
# ChunkSize defines how the mean value is build to handle the drift in rotation data. 
# 120 would build the mean over 120 log entries which results in chunks of 60 seconds
chunkSize = 120
# Rotation threshhold defines which degree has to be reached to count as rotation. 
# Every rotation which overcomes this value will be counted as headrotation
rotation_threshhold = 20.0

# Dash App
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

### Functions

In [3]:
def include_euler_angles(df):
    # Rotation Matrix Phone and Airpods
    mRotPhone = []
    mRotAir = []
    for index, row in df.iterrows():
        rotPhone_first = row[["phoneMotionData.rotationMatrix.m1.1","phoneMotionData.rotationMatrix.m1.2","phoneMotionData.rotationMatrix.m1.3",]]
        rotPhone_second = row[["phoneMotionData.rotationMatrix.m2.1","phoneMotionData.rotationMatrix.m2.2","phoneMotionData.rotationMatrix.m2.3"]]
        rotPhone_third =  row[["phoneMotionData.rotationMatrix.m3.1","phoneMotionData.rotationMatrix.m3.2","phoneMotionData.rotationMatrix.m3.3"]]
        rotPhone_nump = np.array([rotPhone_first.to_numpy(),rotPhone_second.to_numpy(),rotPhone_third.to_numpy()])
        mRotPhone.append(rotPhone_nump)

        rotAir_first = row[["airpodMotionData.rotationMatrix.m1.1","airpodMotionData.rotationMatrix.m1.2","airpodMotionData.rotationMatrix.m1.3",]]
        rotAir_second = row[["airpodMotionData.rotationMatrix.m2.1","airpodMotionData.rotationMatrix.m2.2","airpodMotionData.rotationMatrix.m2.3"]]
        rotAir_third =  row[["airpodMotionData.rotationMatrix.m3.1","airpodMotionData.rotationMatrix.m3.2","airpodMotionData.rotationMatrix.m3.3"]]
        rotAir_nump = np.array([rotAir_first.to_numpy(),rotAir_second.to_numpy(),rotAir_third.to_numpy()])
        mRotAir.append(rotAir_nump)

    # Relative Rotations
    mRotRelatives = []
    for index, rotPhone in enumerate(mRotPhone):
        mRotRelatives.append(rotPhone.dot(mRotAir[index].transpose()))

    # Yaw Values
    relativeYaws = []
    relativeRoll = []
    relativePitch = []
    for index, rotRelative in enumerate(mRotRelatives):
        relativeYaws.append(math.atan2(rotRelative[0,1],rotRelative[0,0])) #Indexes shifted because starting at 0
        relativeRoll.append(math.atan2(rotRelative[1,2],rotRelative[2,2])) #Indexes shifted because starting at 0
        relativePitch.append(- math.asin(rotRelative[0,2]))                #Indexes shifted because starting at 0

    #Convert Radians to Degree
    realtiveYawsDegree = np.degrees(relativeYaws)
    relativeRollDegree = np.degrees(relativeRoll)
    relativePitchDegree = np.degrees(relativePitch)
    
    #import into frame
    df["relativeYaw"] = realtiveYawsDegree.tolist()
    df["relativeRoll"] = relativeRollDegree.tolist()
    df["relativePitch"] = relativePitchDegree.tolist()

def read_data(url):
    with open(url,'r') as f:
        data = json.loads(f.read())
    
    #get infos and delete theme from json
    #infos = data.pop("infos")
    #df["motionTimestampDiff"] = df.apply(lambda x: x["airpodMotionData.timestamp"] - x["phoneMotionData.timestamp"],axis=1)

    # normalize json to readable frame
    df = pd.json_normalize(data,record_path=["timestamps"])

    # convert objects to floats
    df = df.apply(pd.to_numeric, errors='coerce')

    #set timestamp as index
    df.set_index('timestamp',inplace=True)
        
    #Velocity m/s to km/h
    if 'locationData.velocity' in df.columns:
        df["locationData.velocity"] = df["locationData.velocity"].apply(lambda x: x * 3.6)
    #calculate Headrotation
    df["headRotY"] = df.apply(lambda x: x["airpodMotionData.yaw"] - x["phoneMotionData.yaw"],axis=1)
    df["headRotY"] = np.degrees(df["headRotY"]).tolist()
    df["airpodsYawDegrees"] = np.degrees(df["airpodMotionData.yaw"]).tolist()
    df["phoneYawDegrees"] = np.degrees(df["phoneMotionData.yaw"]).tolist()
    
    df["airpodsRollDegrees"] = np.degrees(df["airpodMotionData.roll"]).tolist()
    df["phoneRollDegrees"] = np.degrees(df["phoneMotionData.roll"]).tolist()
    
    df["airpodsPitchDegrees"] = np.degrees(df["airpodMotionData.pitch"]).tolist()
    df["phonePitchDegrees"] = np.degrees(df["phoneMotionData.pitch"]).tolist()
    
    include_euler_angles(df)
    addMeanRotations(df)
    addRotations(df)
    
    return df
def read_srt(url):
    subs = pysrt.open(url)
    return subs
def add_conditions(subject,fig,propertyName): #To-Do use times instead of iterating srts again
    subs = srts[subject]
    shapes = []
    for sub in subs:
        # If Log was writte before video, add the delay to the times to get correct times
        if srt_delays[subject] < 0:
            start = sub.start.minutes * 60 + sub.start.seconds + abs(srt_delays[subject])
            end = sub.end.minutes * 60 + sub.end.seconds + abs(srt_delays[subject])
        else:
            start = sub.start.minutes * 60 + sub.start.seconds - abs(srt_delays[subject])
            end = sub.end.minutes * 60 + sub.end.seconds - abs(srt_delays[subject])
        #display("Condition lasted from {} to {}".format(start,end))#
        color = "#FFFFFF"
        if "Condition Start" in sub.text:
            color = COLOR_SIT_X_HEX
        elif "Condition X" in sub.text:
            color = COLOR_SIT_X_HEX
        elif "Condition A" in sub.text:
            color = COLOR_SIT_A_HEX
        elif "Condition B" in sub.text:
            color = COLOR_SIT_B_HEX
        elif "Condition C" in sub.text:
            color = COLOR_SIT_C_HEX
        elif "Condition D" in sub.text:
            color = COLOR_SIT_D_HEX
        elif "Condition E" in sub.text:
            color = COLOR_SIT_E_HEX
        else:
            color = COLOR_SIT_X_HEX
        shapes.append({
            'type': 'rect',
#            xref: 'x',
#            yref: 'paper',
            'x0': start,
            'y0': subjects[subject][propertyName].min(),
            'x1': end,
            'y1': subjects[subject][propertyName].max(),
            'fillcolor': color,
            'opacity': 0.5,
            'line': {
                'width': 0
            }
        })
    fig.update_layout(shapes = shapes)
def getTimes(subject):
    subs = srts[subject]
    situations = []
    iterations = [0,0,0,0,0]
    for sub in subs:
        # If Log was writte before video, add the delay to the times to get correct times
        if srt_delays[subject] < 0:
            start = sub.start.minutes * 60 + sub.start.seconds + abs(srt_delays[subject])
            end = sub.end.minutes * 60 + sub.end.seconds + abs(srt_delays[subject])
        else:
            start = sub.start.minutes * 60 + sub.start.seconds - abs(srt_delays[subject])
            end = sub.end.minutes * 60 + sub.end.seconds - abs(srt_delays[subject])
        #if "Condition Start" in sub.text:
        #elif "Condition X" in sub.text:
        #elif "Condition D" in sub.text:
        condition = "x"
        iteration = 0
        if "Condition A" in sub.text:
            iterations[0] += 1
            iteration = iterations[0]
            condition = "a"
        elif "Condition B" in sub.text:
            iterations[1] += 1
            iteration = iterations[1]
            condition = "b"
        elif "Condition C" in sub.text:
            iterations[2] += 1
            iteration = iterations[2]
            condition = "c"
        elif "Condition E" in sub.text:
            iterations[4] += 1
            iteration = iterations[4]
            condition = "e"
        else:
            continue
        subjects[subject].loc[((subjects[subject].index >= start) & (subjects[subject].index <= end)),"situation"] = condition
        result = subjects[subject].loc[((subjects[subject].index >= start) & (subjects[subject].index <= end))]
        situations.append((condition,iteration,result.index[0],result.index[-1]))
    return np.array(situations,dtype = ([('situation', 'U10'),('iteration', 'i4'), ('start_index', 'f4'),('end_index', 'f4')]))

def get_meanDuration(times,situation):
    durations = []
    for iteration in times[times["situation"] == situation]:
        durations.append(iteration["end_index"] - iteration["start_index"])
    return np.mean(durations)
def get_sdDuration(times,situation):
    durations = []
    for iteration in times[times["situation"] == situation]:
        durations.append(iteration["end_index"] - iteration["start_index"])
    return np.std(durations)
def splitDf(listElement,df,chunksize):
    df_1 = df.iloc[:chunksize,:]
    df_2 = df.iloc[chunksize:,:]
    listElement.append(df_1)
    return df_2

# Splits the df into chunks (size defined globally) and adds "mean_rotation" for the chunk,wich is the mean of the "relativeYaw"
def addMeanRotations(df):
    splittableDf = df
    chunks = []

    while len(splittableDf) > 0:
        splittableDf = splitDf(chunks,splittableDf,60)

    for chunk_df in chunks:
        df.loc[chunk_df.iloc[0].name:chunk_df.iloc[-1].name,"mean_rotation"] = np.mean(chunk_df["relativeYaw"])

# adds two rows to the dataframe. rotations which indicates the rotation for each row, group_index which indicates to which look count the row is sorted (1 group = 1 look)
def addRotations(df):

    df["rotation"] = "x"
    #Every value which is above the mean_rotation + threshhold is a right headrotation, below its a left headrotation.
    df.loc[(df["relativeYaw"] - df["mean_rotation"] > rotation_threshhold),"rotation"] = "r"
    df.loc[(df["relativeYaw"] - df["mean_rotation"]< rotation_threshhold),"rotation"] = "l"
    df['group_index'] = (df['rotation'] != df['rotation'].shift()).cumsum()
    
#Returns the rotations for each situation for given subject
def getRotations(subject,times):
    rotations = {
        "a" : [],
        "b" : [],
        "c" : [],
        "d" : [],
        "e" : []
    }    
    # For Each situaiton
    for key,values in rotations.items():
        # For each iteration of situation
        for split_sit in times[times["situation"] == key]:
            # group by rotation
            rot_groups = subjects[subject][split_sit["start_index"]:split_sit["end_index"]].groupby(['rotation'])["group_index"]
            # get unqiue groups to count amount
            rots = subjects[subject][split_sit["start_index"]:split_sit["end_index"]].groupby(['rotation'])["group_index"].nunique()
            values.append((rots.get("l" , 0),rots.get("r" , 0)))
    return rotations

def getAngles(subject,times):
    situation_angles = []
    # For every Situation...
    for key in ["a","b","c","d","e"]:
        #for every iteration of this situation
        for index,split_sit in enumerate(times[times["situation"] == key]):
            #group by rotations
            rot_groups = subjects[subject][split_sit["start_index"]:split_sit["end_index"]][["mean_rotation","rotation","relativeYaw","group_index"]]#.groupby(['rotation'])["group_index"]
            rot_groups.groupby(['rotation'])

            # for every found headrotation in iteration
            for group in rot_groups["group_index"].unique():
                df_group = rot_groups.loc[rot_groups["group_index"] == group]
                # left rotation
                if 'l' in df_group["rotation"].unique():
                    #get index of lowest angle in group
                    minIndex = df_group["relativeYaw"].idxmin()
                    #if (df_group.loc[minIndex]["relativeYaw"] - df_group.loc[minIndex]["mean_rotation"]) < -180:
                        #display("Found huge angle from {} with mean {}".format(df_group.loc[minIndex]["relativeYaw"],df_group.loc[minIndex]["mean_rotation"]))
                        #continue
                    situation_angles.append((key,index,"left",df_group.loc[minIndex]["relativeYaw"] - df_group.loc[minIndex]["mean_rotation"]))
                #right rotation
                elif 'r' in df_group["rotation"].unique():
                    #get index of highest angle in group
                    maxIndex = df_group["relativeYaw"].idxmax()
                    #calculate angle in relation to mean rotation
                    #if (df_group.loc[maxIndex]["relativeYaw"] - df_group.loc[maxIndex]["mean_rotation"]) > 180:
                        #display("Found huge angle from {} with mean {}".format(df_group.loc[maxIndex]["relativeYaw"],df_group.loc[maxIndex]["mean_rotation"]))
                        #continue
                    situation_angles.append((key,index,"right",df_group.loc[maxIndex]["relativeYaw"] - df_group.loc[maxIndex]["mean_rotation"]))
    return np.array(situation_angles,dtype = ([('situation', 'U10'),('iteration', 'i4'),('direction', 'U10'), ('angle', 'f4')]))

def getStandtimes(subject,times):
    situation_stands = []
    for situation in ["a","b","c","e"]:
        #for every iteration of this situation
        for index,split_sit in enumerate(times[times["situation"] == situation]):
            df_timeslice = subjects[subject].loc[split_sit["start_index"]:split_sit["end_index"]]["locationData.velocity"]
            group_index = (df_timeslice != df_timeslice.shift()).cumsum()
            # for every found headrotation
            standedInIteration = False
            for stand_sit_group in group_index.unique():
                df_sit = group_index.loc[group_index == stand_sit_group]
                if df_timeslice[df_sit.index[0]] == 0.0:
                    situation_stands.append((situation,index,df_sit.index[-1] - df_sit.index[0]))
                    standedInIteration = True
            if not standedInIteration:
                # This iteration no one stand. Adding 0 to account for in mean calculations
                situation_stands.append((situation,index,0.00))
    return np.array(situation_stands,dtype = ([('situation', 'U10'),('iteration', 'i4'), ('seconds', 'f4')]))
# builds sum over all iterations of one situation, returns mean of this sum
def getMeanOfStandSituation(times,situation):
    sit_times = times[times["situation"] == situation]
    sit_sums = []
    for iteration in np.unique(sit_times["iteration"]):
        sit_sums.append(np.sum(sit_times[sit_times["iteration"] == iteration]["seconds"]))
    return np.mean(sit_sums)
def getSdOfStandSituation(times,situation):
    sit_times = times[times["situation"] == situation]
    sit_sums = []
    for iteration in np.unique(sit_times["iteration"]):
        sit_sums.append(np.sum(sit_times[sit_times["iteration"] == iteration]["seconds"]))
    return np.std(sit_sums)

## Data Read

In [4]:
# Questionaire Data
age = [24,26,22,23,25,28]
#display(np.std(age))

#difficulty,safety
#5 = hard
difficulty = {
    "a" : [5,2,1,2,1,1,2,1],
    "b" : [1,1,1,3,4,1,1,3],
    "c" : [3,1,4,3,3,2,3,2],
    "d" : [1,1,3,3,1,3,1,4],
    "e" : [1,1,2,3,2,2,1,1]
}
safety = {
    "a" : [3,5,4,4,4,4,5,5],
    "b" : [4,5,5,4,2,5,5,4],
    "c" : [4,4,2,3,3,3,5,5],
    "d" : [5,5,4,4,4,4,5,2],
    "e" : [5,5,4,3,3,4,5,5] 
}
means = []
iqrs = []
for situation,values in difficulty.items():
    means.append(np.mean(values))
    q75, q25 = np.percentile(values, [75 ,25])
    iqr = q75 - q25
    iqrs.append(iqr)

#5 = Safe, Flipped Data
means = []
iqrs = []
for situation,values in safety.items():
    means.append(np.mean(values))
    q75, q25 = np.percentile(values, [75 ,25])
    iqr = q75 - q25
    iqrs.append(iqr)

In [5]:
#Subjects
df_subject1 = read_data("./logs/participants/2022-10-17 124155-logfile-subject-1.json")
df_subject2 = read_data("./logs/participants/2022-10-20 111557-logfile-subject-2.json")
#df_subject3 = read_data("./logs/participants/2022-10-25 124003-logfile-subject-3.json")
df_subject3_1 = read_data("./logs/participants/2022-11-04 113121-logfile-subject-3.1.json")
df_subject4 = read_data("./logs/participants/2022-10-25 142229-logfile-subject-4.json")
df_subject5 = read_data("./logs/participants/2022-10-27 165339-logfile-subject-5.json")
df_subject6 = read_data("./logs/participants/2022-10-28 101829-logfile-subject-6.json")
df_subject7 = read_data("./logs/participants/2022-10-28 154837-logfile-subject-7.json")
df_subject8 = read_data("./logs/participants/2022-11-04 122345-logfile-subject-8.json")

#SRTs
srt_subject1 = read_srt("./logs/participants/srts/subs_participant_1.srt")
srt_subject2 = read_srt("./logs/participants/srts/subs_participant_2.srt")
srt_subject3 = read_srt("./logs/participants/srts/subs_participant_3.srt")
srt_subject4 = read_srt("./logs/participants/srts/subs_participant_4.srt")
srt_subject5 = read_srt("./logs/participants/srts/subs_participant_5.srt")
srt_subject6 = read_srt("./logs/participants/srts/subs_participant_6.srt")
srt_subject7 = read_srt("./logs/participants/srts/subs_participant_7.srt")
srt_subject8 = read_srt("./logs/participants/srts/subs_participant_8.srt")

In [6]:
# TO-DO: Streamline into one dictionary
subjects = {
    "Subject 1" : df_subject1,
    "Subject 2" : df_subject2,
    "Subject 3" : df_subject3_1,
    "Subject 4" : df_subject4,
    "Subject 5" : df_subject5,
    "Subject 6" : df_subject6,
    "Subject 7" : df_subject7,
    "Subject 8" : df_subject8
}
srts = {
    "Subject 1" : srt_subject1,
    "Subject 2" : srt_subject2,
    "Subject 3" : srt_subject3,
    "Subject 4" : srt_subject4,
    "Subject 5" : srt_subject5,
    "Subject 6" : srt_subject6,
    "Subject 7" : srt_subject7,
    "Subject 8" : srt_subject8
}
# If negative, the app was running before the camera. Little inconsistency in the study design 
srt_delays = {
    "Subject 1" : - 8.0,
    "Subject 2" : - 29.0,
    "Subject 3" : 12.0 ,
    "Subject 4" : -10.0,
    "Subject 5" : 9.0,
    "Subject 6" : -32.0,
    "Subject 7" : 490.0,
    "Subject 8" : 14.0
}
#[difficulty,safety]
subjects_perceived = {
    "Subject 1" : [[5,1,3,1,1],[3,4,4,5,5]],
    "Subject 2" : [[2,1,1,1,1],[5,5,4,5,5]],
    "Subject 3" : [[1,1,4,3,2],[4,5,2,4,4]],
    "Subject 4" : [[2,3,3,3,3],[4,4,3,4,3]],
    "Subject 5" : [[1,4,3,1,2],[4,2,3,4,3]],
    "Subject 6" : [[1,1,2,3,2],[4,5,3,4,4]],
    "Subject 7" : [[2,1,3,1,1],[5,5,5,5,5]],
    "Subject 8" : [[1,3,2,4,1],[5,4,5,2,5]]
}

# Adding rotations where mean_rotation + threshhold as a identifier field to the dataframes
for df in subjects.values():
    addRotations(df)

# Labels for the plots
available_data_labels = list(subjects.keys())

In [7]:
# Get mean values per subject, per situation
data = []
for key,value in subjects.items():
    times = getTimes(key)
    rotations = getRotations(key,times)
    standtimes = getStandtimes(key,times)
    angles = getAngles(key,times)
    left_angles = angles[(angles["situation"] == key) & (angles["direction"] == "left")]["angle"]
    right_angles = angles[(angles["situation"] == key) & (angles["direction"] == "right")]["angle"]
    data.append([
        key,
        value["relativeYaw"].mean(axis=0),
        value["locationData.velocity"].mean(axis=0),
        value.index[-1],
        get_meanDuration(times,"a"),
        get_meanDuration(times,"b"),
        get_meanDuration(times,"c"),
        0,
        get_meanDuration(times,"e"),
        getMeanOfStandSituation(standtimes,"a"),
        getMeanOfStandSituation(standtimes,"b"),
        getMeanOfStandSituation(standtimes,"c"),
        0,
        getMeanOfStandSituation(standtimes,"e"),
        #Rotations have to be divided by the iterations. We need amount
        np.sum([i[0] for i in rotations["a"]]) / len(rotations["a"]),
        np.sum([i[0] for i in rotations["b"]]) / len(rotations["b"]),
        np.sum([i[0] for i in rotations["c"]]) / len(rotations["c"]),
        0,
        np.sum([i[0] for i in rotations["e"]]) / len(rotations["e"]),
        np.sum([i[1] for i in rotations["a"]]) / len(rotations["a"]),
        np.sum([i[1] for i in rotations["b"]]) / len(rotations["b"]),
        np.sum([i[1] for i in rotations["c"]]) / len(rotations["c"]),
        0,
        np.sum([i[1] for i in rotations["e"]]) / len(rotations["e"]),
        np.mean(angles[(angles["situation"] == "a") & (angles["direction"] == "left")]["angle"]),
        np.mean(angles[(angles["situation"] == "b") & (angles["direction"] == "left")]["angle"]),
        np.mean(angles[(angles["situation"] == "c") & (angles["direction"] == "left")]["angle"]),
        0,
        np.mean(angles[(angles["situation"] == "e") & (angles["direction"] == "left")]["angle"]),
        np.mean(angles[(angles["situation"] == "a") & (angles["direction"] == "right")]["angle"]),
        np.mean(angles[(angles["situation"] == "b") & (angles["direction"] == "right")]["angle"]),
        np.mean(angles[(angles["situation"] == "c") & (angles["direction"] == "right")]["angle"]),
        0,
        np.mean(angles[(angles["situation"] == "e") & (angles["direction"] == "right")]["angle"]),
        #safety
        subjects_perceived[key][1][0],
        subjects_perceived[key][1][1],
        subjects_perceived[key][1][2],
        subjects_perceived[key][1][3],
        subjects_perceived[key][1][4],
        #difficulty
        subjects_perceived[key][0][0],
        subjects_perceived[key][0][1],
        subjects_perceived[key][0][2],
        subjects_perceived[key][0][3],
        subjects_perceived[key][0][4]
    ])
df_all = pd.DataFrame(data, columns=[
    'subject', 
    'mean_rotation',
    'mean_speed',
    "duration",
    'sit_a_duration',
    'sit_b_duration',
    'sit_c_duration',
    'sit_d_duration',
    'sit_e_duration',
    'sit_a_standtime',
    'sit_b_standtime',
    'sit_c_standtime',
    'sit_d_standtime',
    'sit_e_standtime',
    'sit_a_left_rotations',
    'sit_b_left_rotations',
    'sit_c_left_rotations',
    'sit_d_left_rotations',
    'sit_e_left_rotations',
    'sit_a_right_rotations',
    'sit_b_right_rotations',
    'sit_c_right_rotations',
    'sit_d_right_rotations',
    'sit_e_right_rotations',
    'sit_a_left_angle',
    'sit_b_left_angle',
    'sit_c_left_angle',
    'sit_d_left_angle',
    'sit_e_left_angle',
    'sit_a_right_angle',
    'sit_b_right_angle',
    'sit_c_right_angle',
    'sit_d_right_angle',
    'sit_e_right_angle',
    "sit_a_safety",
    "sit_b_safety",
    "sit_c_safety",
    "sit_d_safety",
    "sit_e_safety",
    "sit_a_difficulty",
    "sit_b_difficulty",
    "sit_c_difficulty",
    "sit_d_difficulty",
    "sit_e_difficulty"
])
df_all.loc['Total'] = df_all.mean(numeric_only=True)
#df_all.loc['SD'] = df_all.std(numeric_only=True)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/var/folders/dq/2dn0y4q57nzg9w8bsvw5zv800000gn/T/ipykernel_77339/567311847.py:27: RuntimeWarning: invalid value encountered in scalar divide
  np.sum([i[0] for i in rotations["b"]]) / len(rotations["b"]),
/var/folders/dq/2dn0y4q57nzg9w8bsvw5zv800000gn/T/ipykernel_77339/567311847.py:32: RuntimeWarning: invalid value encountered in scalar divide
  np.sum([i[1] for i in rotations["b"]]) / len(rotations["b"]),
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret

In [8]:
#Correlations
#TO-DOs: Find Empty Slice producer and prevent mean value from empty values
data = []
for key,value in subjects.items():
    times = getTimes(key)
    rotations = getRotations(key,times)
    standtimes = getStandtimes(key,times)
    angles = getAngles(key,times)
    left_angles = angles[(angles["situation"] == key) & (angles["direction"] == "left")]["angle"]
    right_angles = angles[(angles["situation"] == key) & (angles["direction"] == "right")]["angle"]
    data.append([
        key,
        value["relativeYaw"].mean(axis=0),
        value["locationData.velocity"].mean(axis=0),
        value.index[-1],
        get_meanDuration(times,"a"),
        get_meanDuration(times,"b"),
       get_meanDuration(times,"c"),
        0,
        get_meanDuration(times,"e"),
        getMeanOfStandSituation(standtimes,"a"),
        getMeanOfStandSituation(standtimes,"b"),
        getMeanOfStandSituation(standtimes,"c"),
        0,
        getMeanOfStandSituation(standtimes,"e"),
        #Rotations have to be divided by the iterations. We need amount
        [i[0] for i in rotations["a"]],
        [i[0] for i in rotations["b"]],
        [i[0] for i in rotations["c"]],
        0,
        [i[0] for i in rotations["e"]],
        [i[1] for i in rotations["a"]],
        [i[1] for i in rotations["b"]],
        [i[1] for i in rotations["c"]],
        0,
        [i[1] for i in rotations["e"]],
        angles[(angles["situation"] == "a") & (angles["direction"] == "left")]["angle"],
        angles[(angles["situation"] == "b") & (angles["direction"] == "left")]["angle"],
        angles[(angles["situation"] == "c") & (angles["direction"] == "left")]["angle"],
        0,
        angles[(angles["situation"] == "e") & (angles["direction"] == "left")]["angle"],
        angles[(angles["situation"] == "a") & (angles["direction"] == "right")]["angle"],
        angles[(angles["situation"] == "b") & (angles["direction"] == "right")]["angle"],
        angles[(angles["situation"] == "c") & (angles["direction"] == "right")]["angle"],
        0,
        angles[(angles["situation"] == "e") & (angles["direction"] == "right")]["angle"],
        #safety
        subjects_perceived[key][1][0],
        subjects_perceived[key][1][1],
        subjects_perceived[key][1][2],
        subjects_perceived[key][1][3],
        subjects_perceived[key][1][4],
        #difficulty
        subjects_perceived[key][0][0],
        subjects_perceived[key][0][1],
        subjects_perceived[key][0][2],
        subjects_perceived[key][0][3],
        subjects_perceived[key][0][4]
    ])
df_corr = pd.DataFrame(data, columns=[
    'subject', 
    'mean_rotation',
    'mean_speed',
    "duration",
    'sit_a_duration',
    'sit_b_duration',
    'sit_c_duration',
    'sit_d_duration',
    'sit_e_duration',
    'sit_a_standtime',
    'sit_b_standtime',
    'sit_c_standtime',
    'sit_d_standtime',
    'sit_e_standtime',
    'sit_a_left_rotations',
    'sit_b_left_rotations',
    'sit_c_left_rotations',
    'sit_d_left_rotations',
    'sit_e_left_rotations',
    'sit_a_right_rotations',
    'sit_b_right_rotations',
    'sit_c_right_rotations',
    'sit_d_right_rotations',
    'sit_e_right_rotations',
    'sit_a_left_angle',
    'sit_b_left_angle',
    'sit_c_left_angle',
    'sit_d_left_angle',
    'sit_e_left_angle',
    'sit_a_right_angle',
    'sit_b_right_angle',
    'sit_c_right_angle',
    'sit_d_right_angle',
    'sit_e_right_angle',
    "sit_a_safety",
    "sit_b_safety",
    "sit_c_safety",
    "sit_d_safety",
    "sit_e_safety",
    "sit_a_difficulty",
    "sit_b_difficulty",
    "sit_c_difficulty",
    "sit_d_difficulty",
    "sit_e_difficulty"
])
#df_corr.loc['Total'] = df_all.mean(numeric_only=True)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype

In [9]:
# raw values for correlations per situation
data = [
    ["A",
        np.concatenate( df_corr["sit_a_left_rotations"].values, axis=0),
        np.concatenate( df_corr["sit_a_right_rotations"].values, axis=0),
        np.concatenate( df_corr["sit_a_left_angle"].values, axis=0),
        np.concatenate( df_corr["sit_a_right_angle"].values, axis=0),
        df_corr["sit_a_safety"].values,
        df_corr["sit_a_difficulty"].values
    ],
    ["C",
        np.concatenate( df_corr["sit_c_left_rotations"].values, axis=0),
        np.concatenate( df_corr["sit_c_right_rotations"].values, axis=0),
        np.concatenate( df_corr["sit_c_left_angle"].values, axis=0),
        np.concatenate( df_corr["sit_c_right_angle"].values, axis=0),
        df_corr["sit_c_safety"].values,
        df_corr["sit_c_difficulty"].values
    ],
    ["D",
        np.concatenate( df_corr["sit_e_left_rotations"].values, axis=0),
        np.concatenate( df_corr["sit_e_right_rotations"].values, axis=0),
        np.concatenate( df_corr["sit_e_left_angle"].values, axis=0),
        np.concatenate( df_corr["sit_e_right_angle"].values, axis=0),
        df_corr["sit_e_safety"].values,
        df_corr["sit_e_difficulty"].values
    ],
]
df_means_corrs = pd.DataFrame(data, columns=[
    'situation',
    'left_looks',
    'right_looks',
    'left_angles',
    'right_angles',
    'safety',
    'difficulty'
])

In [10]:
left_rots = []
safety_left_rots = []
right_rots = []
safety_right_rots = []
left_angles = []
safety_left_angles = []
right_angles = []
safety_right_angles = []

safety = []
difficulty = []
for index,participant in df_corr.iterrows():
    for situation in ["a","c","e"]:
        #Rotations
        for left_rot in participant["sit_"+situation+"_left_rotations"]:
            left_rots.append(left_rot)
            safety_left_rots.append(participant["sit_"+situation+"_safety"])
        
        for right_rot in participant["sit_"+situation+"_right_rotations"]:
            right_rots.append(right_rot)
            safety_right_rots.append(participant["sit_"+situation+"_safety"])

        #Angles
        for left_angle in participant["sit_"+situation+"_left_angle"]:
            left_angles.append(left_angle)
            safety_left_angles.append(participant["sit_"+situation+"_safety"])
        
        for right_angle in participant["sit_"+situation+"_right_angle"]:
            right_angles.append(right_angle)
            safety_right_angles.append(participant["sit_"+situation+"_safety"])
        
        safety.append(participant["sit_"+situation+"_safety"])
        difficulty.append(participant["sit_"+situation+"_difficulty"])

In [11]:
totals = df_all.loc["Total"]
data = [
    ["A",totals["sit_a_duration"],totals["sit_a_standtime"],0,totals["sit_a_left_rotations"],totals["sit_a_right_rotations"],totals["sit_a_left_angle"],totals["sit_a_right_angle"],totals["sit_a_safety"],totals["sit_a_difficulty"]],
#    ["b",totals["sit_b_duration"],totals["sit_b_standtime"],0,totals["sit_b_left_rotations"],totals["sit_b_right_rotations"],totals["sit_b_left_angle"],totals["sit_b_right_angle"]],
    ["C",totals["sit_c_duration"],totals["sit_c_standtime"],0,totals["sit_c_left_rotations"],totals["sit_c_right_rotations"],totals["sit_c_left_angle"],totals["sit_c_right_angle"],totals["sit_c_safety"],totals["sit_c_difficulty"]],
#    ["d",totals["sit_d_duration"],totals["sit_d_standtime"],0,totals["sit_d_left_rotations"],totals["sit_d_right_rotations"],totals["sit_d_left_angle"],totals["sit_d_right_angle"],totals["sit_d_collisions"]],
    ["D",totals["sit_e_duration"],totals["sit_e_standtime"],0,totals["sit_e_left_rotations"],totals["sit_e_right_rotations"],totals["sit_e_left_angle"],totals["sit_e_right_angle"],totals["sit_e_safety"],totals["sit_e_difficulty"]],
]
df_means_situations = pd.DataFrame(data, columns=[
    'situation',
    'duration', 
    'standtime',
    'speed',
    'left_looks',
    'right_looks',
    'left_angles',
    'right_angles',
    'safety',
    'difficulty'
])

In [12]:
display(df_means_situations)

,situation,duration,standtime,speed,left_looks,right_looks,left_angles,right_angles,safety,difficulty
0,A,54.688465,30.500029,0,4.812500,3.875000,-42.217636,47.636726,4.250,1.875
1,C,34.625759,8.124380,0,2.687500,1.875000,-53.048115,56.336067,3.625,2.625
2,D,22.172016,5.820295,0,2.892361,1.991319,-38.700855,48.721638,4.250,1.625


### Mean Plots

#### Headrotation Angles

In [13]:
fig_angles = make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_yaxes=True, horizontal_spacing=0)
fig_angles.append_trace(go.Bar(x= df_means_situations.left_angles, y=df_means_situations.situation,
                        orientation='h', showlegend=True, 
                        text= np.round(df_means_situations.left_angles,decimals = 0), 
                        name='Left Angles',
                        marker_color= marking_red), 1, 1)
fig_angles.append_trace(go.Bar(x=df_means_situations.right_angles, y=df_means_situations.situation, 
                        orientation='h', showlegend=True,
                        text=np.round(df_means_situations.right_angles,decimals = 0),
                        name='Right Angles', marker_color='#000000'), 1, 2)
fig_angles.update_xaxes(showgrid=False,range=[-58,0],row=1, col=1)
fig_angles.update_xaxes(showgrid=False,range=[0,58],row=1, col=2)
fig_angles.update_yaxes(showgrid=False, categoryorder="category descending", 
                 ticksuffix=' ', showline=False)
fig_angles.update_traces(hovertemplate=None)
fig_angles.update_layout(
#    title='Mean Headrotation Angles per Situation (Field)',
    margin=dict(t=80, b=0, l=70, r=40),
    hovermode="y unified", 
    xaxis_title=' ', yaxis_title=" ",
    width=500, height=300,
    font=dict(family="Lato, sans-serif",size=14),
    legend=dict(orientation="h", yanchor="bottom",y=1, xanchor="center", x=0.5),
    hoverlabel=dict(font_size=13, font_family="Lato, sans-serif"),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

In [14]:
fig_looks = make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_yaxes=True, horizontal_spacing=0)
fig_looks.append_trace(go.Bar(x= - df_means_situations.left_looks, y=df_means_situations.situation,
                        orientation='h', showlegend=True, 
                        text= np.round(df_means_situations.left_looks,decimals = 0), 
                        name='Left Rotations',
                        marker_color= marking_red), 1, 1)
fig_looks.append_trace(go.Bar(x=df_means_situations.right_looks, y=df_means_situations.situation, 
                        orientation='h', showlegend=True,
                        text=np.round(df_means_situations.right_looks,decimals = 0),
                        name='Right Rotations', marker_color='#000000'), 1, 2)
fig_looks.update_xaxes(showgrid=False,range=[-5,0],row=1, col=1)
fig_looks.update_xaxes(showgrid=False,range=[0,5],row=1, col=2)
fig_looks.update_yaxes(showgrid=False, categoryorder="category descending", 
                 ticksuffix=' ', showline=False)
fig_looks.update_traces(hovertemplate=None)
fig_looks.update_layout(
#    title='Mean of Headrotations per Situation(Field)',
    margin=dict(t=80, b=0, l=70, r=40),
    hovermode="y unified", 
    xaxis_title=' ', yaxis_title=" ",
    width=500, height=300,
    font=dict(family="Lato, sans-serif",size=14),
    legend=dict(orientation="h", yanchor="bottom",y=1, xanchor="center", x=0.5),
    hoverlabel=dict(font_size=13, font_family="Lato, sans-serif"),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

In [15]:
### Preceived Safety

In [16]:
fig_duration = go.Figure()
fig_duration.add_trace(go.Bar(x=df_means_situations.safety, y=df_means_situations.situation, 
                        orientation='h', showlegend=True,
                        text=np.round(df_means_situations.safety,decimals = 1),
                        name='Perceived Safety', marker_color= marking_red))
fig_duration.update_xaxes(showgrid=False)
fig_duration.update_yaxes(showgrid=False, categoryorder="category descending",
                 ticksuffix=' ', showline=False)
fig_duration.update_traces(hovertemplate=None)
fig_duration.update_layout(
#    title='Mean of Duration & Standingtimes per Situation (Seconds)(Field)',
    margin=dict(t=80, b=0, l=70, r=40),
    hovermode="y unified", 
    xaxis_title=' ', yaxis_title=" ",
    width=500, height=300,
    font=dict(family="Lato, sans-serif",size=14),
    legend=dict(orientation="h", yanchor="bottom",y=1, xanchor="center", x=0.5),
    hoverlabel=dict(font_size=13, font_family="Lato, sans-serif"),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)
display(fig_duration)

#### Duration per Subject

In [17]:
fig_duration = go.Figure()
fig_duration.add_trace(go.Bar(x=df_means_situations.duration, y=df_means_situations.situation, 
                        orientation='h', showlegend=True,
                        text=np.round(df_means_situations.duration,decimals = 0),
                        name='Duration', marker_color= marking_red))
fig_duration.add_trace(go.Bar(x=df_means_situations.standtime, y=df_means_situations.situation, 
                        orientation='h', showlegend=True,
                        text=np.round(df_means_situations.standtime,decimals = 0),
                        name='Standtime', marker_color='#000000'))
fig_duration.update_xaxes(showgrid=False)
fig_duration.update_yaxes(showgrid=False, categoryorder="category descending",
                 ticksuffix=' ', showline=False)
fig_duration.update_traces(hovertemplate=None)
fig_duration.update_layout(
#    title='Mean of Duration & Standingtimes per Situation (Seconds)(Field)',
    margin=dict(t=80, b=0, l=70, r=40),
    hovermode="y unified", 
    xaxis_title=' ', yaxis_title=" ",
    width=500, height=300,
    font=dict(family="Lato, sans-serif",size=14),
    legend=dict(orientation="h", yanchor="bottom",y=1, xanchor="center", x=0.5),
    hoverlabel=dict(font_size=13, font_family="Lato, sans-serif"),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

In [18]:
fig_duration_subject = go.Figure()
fig_duration_subject.add_trace(go.Bar(x = df_all.duration, y = df_all.subject, 
                        orientation='h', showlegend=True,
                        text=np.round(df_all.duration,decimals = 0),
                        name='Duration', marker_color=marking_red))
fig_duration_subject.update_xaxes(showgrid=False)
fig_duration_subject.update_yaxes(showgrid=False, categoryorder='array', categoryarray= ["Subject 12","Subject 11","Subject 10","Subject 9","Subject 8","Subject 7","Subject 6","Subject 5","Subject 4","Subject 3","Subject 2","Subject 1"],
                 ticksuffix=' ', showline=False)
fig_duration_subject.update_traces(hovertemplate=None)
fig_duration_subject.update_layout(
#    title='Duration per Subject (Seconds)(Field)',
    margin=dict(t=80, b=0, l=70, r=40),
    hovermode="y unified", 
    xaxis_title=' ', yaxis_title=" ",
    width=500, height=400,
    font=dict(family="Lato, sans-serif",size=14),
    legend=dict(orientation="h", yanchor="bottom",y=1, xanchor="center", x=0.5),
    hoverlabel=dict(font_size=13, font_family="Lato, sans-serif"),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)

### Correlations

In [19]:
# Spearman correlations and p-values
display("Difficulty {}".format(spearmanr(difficulty,safety)))
display("Left Looks {}".format(spearmanr(safety_left_rots, left_rots)))
display("Right Looks {}".format(spearmanr(safety_right_rots, right_rots)))
display("Left angles {}".format(spearmanr(safety_left_angles, left_angles)))
display("Right Angles {}".format(spearmanr(safety_right_angles, right_angles)))

'Difficulty SpearmanrResult(correlation=-0.5721574955484239, pvalue=0.003484415825772726)'

'Left Looks SpearmanrResult(correlation=0.029454253694129223, pvalue=0.7711258684692133)'

'Right Looks SpearmanrResult(correlation=0.024260619285036342, pvalue=0.8106471202430714)'

'Left angles SpearmanrResult(correlation=-0.12658132809506686, pvalue=0.025122891964864034)'

'Right Angles SpearmanrResult(correlation=0.04328015650171581, pvalue=0.5192926884058173)'

In [20]:
# Plots for correlations. Trendline generated as linear trendline via https://plotly.com/python/linear-fits/
fig_corr_diff = go.Figure(layout=layout)
fig_corr_diff.add_trace(go.Scatter(mode='markers',x = safety,y = difficulty,showlegend=False,name='Safety X Difficulty', marker_color='#000000'))
trend_difficulty = px.scatter(y=difficulty, x= safety, trendline="ols")
trendline = trend_difficulty.data[1]
trendline.marker.color = marking_red
fig_corr_diff.add_trace(trendline)

#Rotations
fig_corr_rleft = go.Figure(layout=layout)
fig_corr_rleft.add_trace(go.Scatter(mode='markers',x = safety_left_rots,y = left_rots,showlegend=False,name='Safety X Left Rotations', marker_color='#000000'))
trend_rotations = fig = px.scatter(y=left_rots, x= safety_left_rots, trendline="ols")
trendline = trend_rotations.data[1]
trendline.marker.color = marking_red
fig_corr_rleft.add_trace(trendline)

fig_corr_rRight = go.Figure(layout=layout)
fig_corr_rRight.add_trace(go.Scatter(mode='markers',x = safety_right_rots,y = right_rots,showlegend=False,name='Safety X Left Rotations', marker_color='#000000'))
trend_rotations = fig = px.scatter(y=right_rots, x= safety_right_rots, trendline="ols")
trendline = trend_rotations.data[1]
trendline.marker.color = marking_red
fig_corr_rRight.add_trace(trendline)

#Angles
fig_corr_aLeft = go.Figure(layout=layout)
fig_corr_aLeft.add_trace(go.Scatter(mode='markers',x = safety_left_angles,y = left_angles,showlegend=False,name='Safety X Left Angles', marker_color='#000000'))
trend_angles = fig = px.scatter(y=left_angles, x= safety_left_angles, trendline="ols")
trendline = trend_angles.data[1]
trendline.marker.color = marking_red
fig_corr_aLeft.add_trace(trendline)

fig_corr_aRight = go.Figure(layout=layout)
fig_corr_aRight.add_trace(go.Scatter(mode='markers',x = safety_right_angles,y = right_angles,showlegend=False,name='Safety X Left Angles', marker_color='#000000'))
trend_angles = fig = px.scatter(y=right_angles, x= safety_right_angles, trendline="ols")
trendline = trend_angles.data[1]
trendline.marker.color = marking_red
fig_corr_aRight.add_trace(trendline)


yaxis = ["Difficulty","Left Rotations","Right Rotations","Left Angles","Right Angles"]
yFormat = ["","","","°","°"]
for index,fig in enumerate([fig_corr_diff,fig_corr_rleft,fig_corr_rRight,fig_corr_aLeft,fig_corr_aRight]):
    fig.update_layout(
    #    title='Mean of Headrotations per Situation(Field)',
        margin=dict(t=20, b=0, l=70, r=40),
        hovermode="y unified", 
        xaxis_title='Safety', 
        xaxis_dtick = 1,
        yaxis_title=yaxis[index],
        yaxis_ticksuffix = yFormat[index],
        width=600, height=400,
        font=dict(family="Lato, sans-serif",size=14),
        legend=dict(orientation="h", yanchor="bottom",y=1, xanchor="center", x=0.5),
        hoverlabel=dict(font_size=13, font_family="Lato, sans-serif"))
    display(fig)

## App Layout

In [21]:
app.layout = html.Div([
    html.Div([dcc.Markdown('''### Realworld Study''',id='headline_subject')],style={'width': '100%', 'display': 'block'}),
    html.Div([dcc.Markdown('''#### General Plots 
    > This data represents the mean values from ALL Subjects''')],style={'width': '100%', 'display': 'block'}),
    html.Div([dcc.Graph(figure = fig_looks)], style={'width': '40%', 'display': 'inline-block'}),
    html.Div([dcc.Graph(figure = fig_angles)], style={'width': '40%', 'display': 'inline-block'}),
    html.Div([dcc.Graph(figure = fig_duration)], style={'width': '40%', 'display': 'inline-block'}),
    html.Div([dcc.Graph(figure = fig_duration_subject)], style={'width': '40%', 'display': 'inline-block'}),
        html.Div([dcc.Markdown('''#### Subject Specific Plots
    > This data represents the selected subject only''')],style={'width': '100%', 'display': 'block'}),
    html.Div([
        html.Label('Subject'),
        dcc.Dropdown(
            id='filter_subject',
            options=[{'label': i, 'value': i} for i in available_data_labels],
            value='Choose Subject '
    )],style={'width': '100%', 'display': 'inline-block'}),
    html.Div([
        html.Label('Timestamp (Seconds)'),
        dcc.Slider(
            id='filter-slider-timestamp',
            min=0,
            max=0,
            value=0,
            tooltip={"placement": "bottom", "always_visible": True},
            step=60
        )
    ], style={'width': '100%', 'display': 'inline-block'}),
    html.Div([
        html.Label('NO Condition',style={'backgroundColor' : COLOR_SIT_X_HEX,'color': "#FFFFFF",'borderRadius':'5px','marginLeft':'10px','padding': '10px'}),
        html.Label('Condition A',style={'backgroundColor' : COLOR_SIT_A_HEX,'color': "#FFFFFF",'borderRadius':'5px','marginLeft':'10px','padding': '10px'}),
        html.Label('Condition B',style={'backgroundColor' : COLOR_SIT_B_HEX,'color': "#FFFFFF",'borderRadius':'5px','marginLeft':'10px','padding': '10px'}),
        html.Label('Condition C',style={'backgroundColor' : COLOR_SIT_C_HEX,'color': "#FFFFFF",'borderRadius':'5px','marginLeft':'10px','padding': '10px'}),
        html.Label('Condition D',style={'backgroundColor' : COLOR_SIT_D_HEX,'color': "#FFFFFF",'borderRadius':'5px','marginLeft':'10px','padding': '10px'}),
        html.Label('Condition E',style={'backgroundColor' : COLOR_SIT_E_HEX,'color': "#FFFFFF",'borderRadius':'5px','marginLeft':'10px','padding': '10px'}),
    ], style={'width': '100%', 'display': 'inline-block'}),
    html.Div([dcc.Graph(id='headmovement-scatter',hoverData={'points': [{'x': 0}]})], style={'width': '100%', 'display': 'block'}),
    html.Div([dcc.Graph(id='gps-coords')], style={'display': 'inline-block', 'width': '100%'}),
    html.Div([dcc.Graph(id='yaws-scatter',hoverData={'points': [{'x': 0}]})], style={'width': '100%', 'display': 'block'}),
    html.Div([dcc.Graph(id='rolls-scatter',hoverData={'points': [{'x': 0}]})], style={'width': '100%', 'display': 'block'}),
    html.Div([dcc.Graph(id='pitchs-scatter',hoverData={'points': [{'x': 0}]})], style={'width': '100%', 'display': 'block'}),
    html.Div([dcc.Graph(id='velocity')], style={'display': 'inline-block', 'width': '50%'}),
    html.Div([dcc.Graph(id='acceleration')], style={'display': 'inline-block', 'width': '50%'}),
    html.Div([dcc.Graph(id='altitude')], style={'display': 'inline-block', 'width': '50%'}),
    html.Div([dcc.Graph(id='phone_battery')], style={'display': 'inline-block', 'width': '50%'}),
    #html.Div([dcc.Graph(id='headMesh')], style={'display': 'inline-block', 'width': '25%'}),
])

## Live Interactions

### Timestamps

In [22]:
@app.callback(
    dash.dependencies.Output('filter-slider-timestamp', 'value'),
    [dash.dependencies.Input('filter_subject', 'value'),
    dash.dependencies.Input('headmovement-scatter', 'hoverData')])
def update_timestampValue(subject,hoverdata):
    if not subject in subjects:
        return 0
    # TO-DO!! -> Timestamp und steps dynamisch holen
    lastTimestamp = float(subjects[subject].iloc[-1].name)
    #display("The experiment from subject "+infos["subject"]+" lasted " + str(lastTimestamp) + " seconds and started at " + infos["starttime"])
    halfminuteSteps = np.arange(0,lastTimestamp,30)
    halfminuteStepLabels = list(map(lambda x: str(int(x)), halfminuteSteps))
    if "x" in hoverdata['points'][0]:
        return hoverdata['points'][0]["x"]
    else:
        return 0
@app.callback(
    dash.dependencies.Output('filter-slider-timestamp', 'min'),
    [dash.dependencies.Input('filter_subject', 'value')])
def update_timestampMin(subject):
    if not subject in subjects:
        return 0
    return subjects[subject].index.min()
@app.callback(
    dash.dependencies.Output('filter-slider-timestamp', 'max'),
    [dash.dependencies.Input('filter_subject', 'value')])
def update_timestampMax(subject):
    if not subject in subjects:
        return 0
    return subjects[subject].index.max()

### Headmovement

In [23]:
@app.callback(
    dash.dependencies.Output('headmovement-scatter', 'figure'),
    [dash.dependencies.Input('filter_subject', 'value')])
def update_headrotation(subject):
    if not subject in subjects:
        return go.Figure()
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=subjects[subject].index,
        y=subjects[subject]["relativeYaw"],
        mode="lines",
        name="Headrotation (relative Yaw)")
    )
#    fig.add_annotation(
#        x=timestamp, 
#        y=subjects[subject]["relativeYaw"].max(),
#        text="Chosen Time " + str(timestamp),
#        showarrow=False,
#        yshift=10
#    )
    fig.update_layout(
        title_text="Headmovement",
        autosize=True,
        hovermode='closest'
        # Optional visualization of threshhold
        #shapes = [
            # Line Horizontal
        #    {
        #        'type': 'line',
        #        'x0': subjects[subject].index.min(),
        #        'y0': -rotation_threshhold,
        #        'x1': subjects[subject].index.max(),
        #        'y1': -rotation_threshhold,
        #        'line': {
        #            'color': 'rgb(50, 171, 96)',
        #            'width': 2,
        #            'dash' : "dot"
        #        },
        #    },
        #    {
        #        'type': 'line',
        #        'x0': subjects[subject].index.min(),
        #        'y0': rotation_threshhold,
        #        'x1': subjects[subject].index.max(),
        #        'y1': rotation_threshhold,
        #        'line': {
        #            'color': 'rgb(50, 171, 96)',
        #            'width': 2,
        #            'dash' : "dot"
        #        },
        #    }
        #]
    )
    #add_conditions(subject,fig,"relativeYaw")
    fig.update_xaxes(title_text='Time(seconds)')
    fig.update_yaxes(title_text='Headrotation (Degrees)')

    return fig

#### GPS Coordinates

In [24]:
@app.callback(
    dash.dependencies.Output('gps-coords', 'figure'),
    [dash.dependencies.Input('filter_subject', 'value'),
     dash.dependencies.Input('filter-slider-timestamp', 'value')])
def update_gps_coords(subject, timestamp):
    if not subject in subjects:
        return go.Figure()
    
    fig = go.Figure()
    print("got timestamp " + str(timestamp))
    #Add all Points
    fig.add_trace(go.Scattermapbox(
        lon = subjects[subject]["locationData.longitude"],
        lat = subjects[subject]["locationData.latitude"],
        mode = 'lines',
        name = "Track"
    ))

    #Highlight current point
    fig.add_trace(go.Scattermapbox(
        lon=subjects[subject].iloc[[timestamp]]["locationData.longitude"],
        lat=subjects[subject].iloc[[timestamp]]["locationData.latitude"],
        mode = 'markers',
        name = "Position at Timestamp",
        marker = go.scattermapbox.Marker(
            size = 12,
            color = 'rgb(50, 171, 96)',
            opacity = 0.9,
            #symbol = "bicycle"
        )
    ))

    fig.update_layout(
        title_text = "GPS Coordinates",
        autosize = True,
        hovermode = 'closest',
        mapbox = dict(
            accesstoken = "pk.eyJ1Ijoibm9tYW5kZXMiLCJhIjoiY2w3cmticXRhMGc0OTN3cDJ5Y2pqcnZ3YiJ9.R8snls4l3IE8Z2cgQ5Ti3g",
            #bearing=0,
            #pitch=0,
            center = dict(
                lat = subjects[subject].iloc[subjects[subject].shape[0]//2]["locationData.latitude"], #middle point of the log
                lon = subjects[subject].iloc[subjects[subject].shape[0]//2]["locationData.longitude"]
            ),
            zoom=13.5
    ))

    return fig

#### Yaws

In [25]:
@app.callback(
    dash.dependencies.Output('yaws-scatter', 'figure'),
    [dash.dependencies.Input('filter_subject', 'value'),
     dash.dependencies.Input('filter-slider-timestamp', 'value')])
def update_yaws(subject, timestamp):
    if not subject in subjects:
        return go.Figure()
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=subjects[subject].index,
        y=subjects[subject]["relativeYaw"],
        mode="lines",
        name="Relative Yaw")
    )
    fig.add_trace(go.Scatter(
        x=subjects[subject].index,
        y=subjects[subject]["airpodsYawDegrees"],
        mode="lines",
        name="Raw Yaw Airpods")
    )
    fig.add_trace(go.Scatter(
        x=subjects[subject].index,
        y=subjects[subject]["phoneYawDegrees"],
        mode="lines",
        name="Raw Yaw iPhone")
    )
    fig.add_annotation(
        x=timestamp, 
        y=subjects[subject]["relativeYaw"].max(),
        text="Chosen Time " + str(timestamp),
        showarrow=False,
        yshift=10
    )
    fig.update_layout(
        title_text="Yaws",
        autosize=True,
        hovermode='closest',
        shapes = [
            # Line Horizontal
            {
                'type': 'line',
                'x0': timestamp,
                'y0': subjects[subject]["relativeYaw"].min(),
                'x1': timestamp,
                'y1': subjects[subject]["relativeYaw"].max(),
                'line': {
                    'color': 'rgb(50, 171, 96)',
                    'width': 2
                },
            }
        ]
    )
    add_conditions(subject,fig,"airpodsYawDegrees")
    fig.update_xaxes(title_text='Time(seconds)')
    fig.update_yaxes(title_text='Yaw (Degrees)')

    return fig

#### Rolls

In [26]:
@app.callback(
    dash.dependencies.Output('rolls-scatter', 'figure'),
    [dash.dependencies.Input('filter_subject', 'value'),
     dash.dependencies.Input('filter-slider-timestamp', 'value')])
def update_rolls(subject, timestamp):
    if not subject in subjects:
        return go.Figure()
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=subjects[subject].index,
        y=subjects[subject]["relativeRoll"],
        mode="lines",
        name="Relative Roll")
    )
    fig.add_trace(go.Scatter(
        x=subjects[subject].index,
        y=subjects[subject]["airpodsRollDegrees"],
        mode="lines",
        name="Raw Roll Airpods")
    )
    fig.add_trace(go.Scatter(
        x=subjects[subject].index,
        y=subjects[subject]["phoneRollDegrees"],
        mode="lines",
        name="Raw Roll iPhone")
    )
    fig.add_annotation(
        x=timestamp, 
        y=subjects[subject]["relativeRoll"].max(),
        text="Chosen Time " + str(timestamp),
        showarrow=False,
        yshift=10
    )
    fig.update_layout(
        title_text="Rolls (Body/Bike Tilt Angle ?)",
        autosize=True,
        hovermode='closest',
        shapes = [
            # Line Horizontal
            {
                'type': 'line',
                'x0': timestamp,
                'y0': subjects[subject]["relativeRoll"].min(),
                'x1': timestamp,
                'y1': subjects[subject]["relativeRoll"].max(),
                'line': {
                    'color': 'rgb(50, 171, 96)',
                    'width': 2
                },
            }
        ]
    )
    add_conditions(subject,fig,"airpodsRollDegrees")
    fig.update_xaxes(title_text='Time(seconds)')
    fig.update_yaxes(title_text='Roll (Degrees)')

    return fig

#### Pitchs

In [27]:
@app.callback(
    dash.dependencies.Output('pitchs-scatter', 'figure'),
    [dash.dependencies.Input('filter_subject', 'value'),
     dash.dependencies.Input('filter-slider-timestamp', 'value')])
def update_pitchs(subject, timestamp):
    if not subject in subjects:
        return go.Figure()
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=subjects[subject].index,
        y=subjects[subject]["relativePitch"],
        mode="lines",
        name="Relative Pitch")
    )
    fig.add_trace(go.Scatter(
        x=subjects[subject].index,
        y=subjects[subject]["airpodsPitchDegrees"],
        mode="lines",
        name="Raw Pitch Airpods")
    )
    fig.add_trace(go.Scatter(
        x=subjects[subject].index,
        y=subjects[subject]["phonePitchDegrees"],
        mode="lines",
        name="Raw Pitch iPhone")
    )
    fig.add_annotation(
        x=timestamp, 
        y=subjects[subject]["relativePitch"].max(),
        text="Chosen Time " + str(timestamp),
        showarrow=False,
        yshift=10
    )
    fig.update_layout(
        title_text="Pitchs (Bike Altitude / Look at Phone ?)",
        autosize=True,
        hovermode='closest',
        shapes = [
            # Line Horizontal
            {
                'type': 'line',
                'x0': timestamp,
                'y0': subjects[subject]["relativePitch"].min(),
                'x1': timestamp,
                'y1': subjects[subject]["relativePitch"].max(),
                'line': {
                    'color': 'rgb(50, 171, 96)',
                    'width': 2
                },
            }
        ]
    )
    add_conditions(subject,fig,"airpodsPitchDegrees")
    fig.update_xaxes(title_text='Time(seconds)')
    fig.update_yaxes(title_text='Pitch (Degrees)')

    return fig

#### Altitude

In [28]:
@app.callback(
    dash.dependencies.Output('altitude', 'figure'),
    [dash.dependencies.Input('filter_subject', 'value'),
     dash.dependencies.Input('filter-slider-timestamp', 'value')])
def update_altitude(subject, timestamp):
    if not subject in subjects:
        return go.Figure()
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=subjects[subject].index,
        y=subjects[subject]["locationData.altitude"],
        mode="lines",fill="tozeroy")
    )
    fig.add_annotation(
        x=timestamp, 
        y=subjects[subject]["locationData.altitude"].max(),
        text="Chosen Time " + str(timestamp),
        showarrow=False,
        yshift=10
    )
    fig.update_layout(
        title_text="Altitude",
        autosize=True,
        hovermode='closest',
        shapes = [
        # Line Horizontal
        {
            'type': 'line',
            'x0': timestamp,
            'y0': subjects[subject]["locationData.altitude"].min(),
            'x1': timestamp,
            'y1': subjects[subject]["locationData.altitude"].max(),
            'line': {
                'color': 'rgb(50, 171, 96)',
                'width': 2
            },
        }]
    )
    add_conditions(subject,fig,"locationData.altitude")
    fig.update_xaxes(title_text='Time (seconds)')
    fig.update_yaxes(
        title_text='Altitude (meters)',
        range = [subjects[subject]["locationData.altitude"].min(),subjects[subject]["locationData.altitude"].max()]
    )

    return fig

#### Acceleration

In [29]:
@app.callback(
    dash.dependencies.Output('acceleration', 'figure'),
    [dash.dependencies.Input('filter_subject', 'value'),
     dash.dependencies.Input('filter-slider-timestamp', 'value')])
def update_acceleration(subject, timestamp):
    if not subject in subjects:
        return go.Figure()
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=subjects[subject].index,
        y=subjects[subject]["phoneAcceleration.y"],
        mode="lines")
    )
    fig.add_annotation(
        x=timestamp, 
        y=subjects[subject]["phoneAcceleration.y"].max(),
        text="Chosen Time " + str(timestamp),
        showarrow=False,
        yshift=10
    )
    fig.update_layout(
        title_text="Acceleration",
        autosize=True,
        hovermode='closest',
        shapes = [
        # Line Horizontal
        {
            'type': 'line',
            'x0': timestamp,
            'y0': subjects[subject]["phoneAcceleration.y"].min(),
            'x1': timestamp,
            'y1': subjects[subject]["phoneAcceleration.y"].max(),
            'line': {
                'color': 'rgb(50, 171, 96)',
                'width': 2
            },
        }]
    )
    add_conditions(subject,fig,"phoneAcceleration.y")
    fig.update_xaxes(title_text='Time (seconds)')
    fig.update_yaxes(title_text='Acceleration Y')
    return fig

#### Velocity

In [30]:
@app.callback(
    dash.dependencies.Output('velocity', 'figure'),
    [dash.dependencies.Input('filter_subject', 'value'),
     dash.dependencies.Input('filter-slider-timestamp', 'value')])
def update_velocity(subject, timestamp):
    if not subject in subjects:
        return go.Figure()
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=subjects[subject].index,
        y=subjects[subject]["locationData.velocity"],
        mode="lines")
    )
    fig.add_annotation(
        x=timestamp, 
        y=subjects[subject]["locationData.velocity"].max(),
        text="Chosen Time " + str(timestamp),
        showarrow=False,
        yshift=10
    )
    fig.update_layout(
        title_text="Velocity",
        autosize=True,
        hovermode='closest',
        shapes = [
        # Line Horizontal
        {
            'type': 'line',
            'x0': timestamp,
            'y0': subjects[subject]["locationData.velocity"].min(),
            'x1': timestamp,
            'y1': subjects[subject]["locationData.velocity"].max(),
            'line': {
                'color': 'rgb(50, 171, 96)',
                'width': 2
            },
        }]
    )
    add_conditions(subject,fig,"locationData.velocity")
    fig.update_xaxes(title_text='Time (seconds)')
    fig.update_yaxes(title_text='Velocity (km/h)')

    return fig

#### Headline

In [31]:
@app.callback(
    dash.dependencies.Output('headline_subject', 'children'),
    [dash.dependencies.Input('filter_subject', 'value')])
def update_headline(subject):
    return "### Realworld Study - " + subject

#### Phone Battery

In [32]:
@app.callback(
    dash.dependencies.Output('phone_battery', 'figure'),
    [dash.dependencies.Input('filter_subject', 'value')])
def update_battery(subject):
    if not subject in subjects:
        return go.Figure()
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=subjects[subject].index, 
        y=subjects[subject]["phoneBattery"],
        fill="tozeroy")
    )
    fig.add_annotation(
        x=timestamp, 
        y=subjects[subject]["phoneBattery"].max(),
        text="Chosen Time " + str(timestamp),
        showarrow=False,
        yshift=10
    )
    fig.update_layout(
        title_text="Phone Battery",
        autosize=True,
        hovermode='y unified',
        shapes = [
        # Line Horizontal
        {
            'type': 'line',
            'x0': timestamp,
            'y0': subjects[subject]["phoneBattery"].min(),
            'x1': timestamp,
            'y1': subjects[subject]["phoneBattery"].max(),
            'line': {
                'color': 'rgb(50, 171, 96)',
                'width': 2
            },
        }]
    )
    fig.update_xaxes(title_text='Time (seconds)')
    fig.update_yaxes(title_text='Battery level %')

    return fig

### Server Start

In [33]:
app.run_server(mode='external', port = 8090, dev_tools_ui=True, debug=True,dev_tools_hot_reload =True, threaded=True)

Dash app running on http://127.0.0.1:8090/


127.0.0.1 - - [12/Jan/2023 13:51:46] code 400, message Bad request version ('\x81Ôw]õøëÓRP\x98ÚrsÇªê\x19¿ýÛ)¯Ë¥ð.¼\x11\x97Ð{\x00*ªª\x13\x01\x13\x02\x13\x03À,À+Ì©À0À/Ì¨À')
127.0.0.1 - - [12/Jan/2023 13:51:46] code 400, message Bad request version ('\x9de7\x889Ñ\x99DÓ\x80ÿ\x94aoTÕz\x7f')
127.0.0.1 - - [12/Jan/2023 13:51:46] code 400, message Bad request syntax ('\x16\x03\x01\x00}\x01\x00\x00y\x03\x01Ø#£3Ì±vã/5*CPkÓ"\x14îí\x16¬·W¹¹²\x7fËÏdï/\x00\x00\x14À')
